In [3]:
import tensorflow as tf
import numpy as np
import random
import copy
from konlpy.tag import Mecab; m = Mecab()

# 데이터 로드, 전처리 

In [4]:
with open('./data/dialogue_sample.txt', 'r', encoding='utf-8') as f:
    raw = f.readlines()

In [5]:
conv=[]
for r in raw:
    if r !='\n':
        conv.append(r)

In [6]:
pX = [r.split('\\t')[0] for r in conv]
py = [r.split('\\t')[1][:-1] for r in conv]

In [7]:
def one_hot(x,Vocab_size):
    return np.identity(Vocab_size)[x:x+1]

In [8]:
def vocab_encode(text, vocab):
    """
    text가 Vocab(characters) 안에 있으면 index로 변환
    """
    return [vocab.index(x) + 1 for x in text if x in vocab]

def vocab_decode(array, vocab):
    """
    index를 다시 문자열로 디코딩
    """
    return ''.join([vocab[x - 1] for x in array])


In [9]:
def posTag(sentences):
    result = []
    for sent in sentences:
        temp = m.morphs(sent)
        result.append(temp)
    return result

In [10]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0)

In [11]:
flatten = lambda l: [item for sublist in l for item in sublist]

In [12]:
X = posTag(pX)
y = posTag(py)

# Build Vocab

In [13]:
tokenset = flatten(X+y)

In [14]:
tokenset.extend(['<UNK>','<NUM>','<PAD>'])

In [15]:
tokenset = list(set(tokenset))

In [16]:
len(tokenset)

211

# Padding and One-hot 

In [17]:
origin_X = copy.deepcopy(X)

In [18]:
origin_X = [' '.join(sent) for sent in origin_X]

In [19]:
for x in X:
    left = 10-len(x)
    x.extend(['<PAD>']*left)

In [20]:
X_vector = [vocab_encode(x,tokenset) for x in X]

In [21]:
y_vector = [vocab_encode(y_,tokenset) for y_ in y]

In [22]:
len(X_vector)

68

In [23]:
HIDDEN = 20
DIM = 20 # input dim
T = 10
Vocab = len(tokenset)+1
Batch = len(X_vector)

In [24]:
for x in range(len(X_vector)):
    for j in range(len(X_vector[x])):
        X_vector[x][j] = one_hot(X_vector[x][j],Vocab)

In [25]:
for y_ in range(len(y_vector)):
    for j in range(len(y_vector[y_])):
        y_vector[y_][j] = one_hot(y_vector[y_][j],Vocab)

# Batch 형태로 만들기 

In [26]:
X_vector = [np.array(x) for x in X_vector]
py_vector = [np.array(y_) for y_ in y_vector]

In [27]:
X_vector = np.squeeze(X_vector)

In [28]:
y_vector=[]
for y_ in py_vector:
    if len(y_) == 1:
        continue
    
    y_vector.append(np.squeeze(y_))

In [29]:
X_vector = np.reshape(X_vector,(T,Batch,Vocab))

In [30]:
X_vector[0].shape

(68, 212)

#  Encoder

In [47]:
from numpy import linalg as LA

In [53]:
max_global_norm = 15000

In [31]:
X_vector.shape

(10, 68, 212)

In [35]:
EB = np.random.randn(Vocab,DIM) # input one-hot vector -> Embedding size vector로 projection하는 matrix 
Whx = np.random.randn(HIDDEN,DIM)
Whh = np.random.randn(HIDDEN,HIDDEN)
Ws = np.random.randn(Vocab,HIDDEN)

In [36]:
np.dot(X_vector[0],EB).shape

(68, 20)

In [106]:
hs = {} # time-step 동안의 hidden state
ss = {} # time-step 동안의 Whh*h_{t-1}
xs = {} # time-step 동안의 Whx*x_t
hs[-1] = np.array([np.random.randn(HIDDEN)]*Batch).T # 초기 hidden state
for t in range(T):
    x = np.dot(X_vector[t],EB) # embedding lookup
    xs[t] = np.dot(Whx,x.T)
    ss[t] = np.dot(Whh,hs[t-1])
    hs[t] = np.maximum(0, ss[t]+xs[t]) # ReLU(Whh*h_{t-1} + Whx*x_t)
    #norm = LA.norm(hs[t])
    #if norm > max_global_norm:
    #    hs[t] = (max_global_norm/norm)*hs[t]

### Context vector 

In [107]:
hs[-1].shape # context vector

(20, 68)

# Decoder 

In [108]:
dch = {}
ds = {}
oy = []
dch[-1] = hs[-1]
for t in range(T):
    dch[t] = np.dot(Whh,dch[t-1])
    #norm = LA.norm(dch[t])
    #if norm > max_global_norm:
    #    dch[t] = (max_global_norm/norm)*dch[t]
    hy = np.maximum(0,np.dot(Whh,dch[t-1]))
    oy.append(softmax(np.dot(Ws,hy)))

/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:3: RuntimeWarning: overflow encountered in exp
  app.launch_new_instance()
/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:3: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()


### Sample 문장 하나 decode 

In [109]:
oy = np.array(oy)

In [110]:
oy = np.reshape(oy,(Batch,T,Vocab))

In [111]:
answer = ''
index = random.choice(range(Batch))
for i in range(len(y[index])):
    answer+=tokenset[np.argmax(oy[index][i])]
    
print(answer)

야야니음날음별론데


# Backprop 

In [87]:
def cross_entropy(hypo,label):
    
    total_loss = 0
    for i in range(len(label)):
        instance_loss = 0
        for j in range(len(label[i])):
            instance_loss += -np.sum(label[i][j]*np.log(hypo[i][j]))
        total_loss+=instance_loss
    
    return total_los3

In [67]:
oy[3].shape

(10, 212)

In [68]:
y_vector[10].shape

(2, 212)

In [89]:
cross_entropy(oy,y_vector)

/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:7: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:7: RuntimeWarning: invalid value encountered in multiply


nan

In [ ]:
# backward pass of the RNN
dhs = {}
dss = {}
dhs[T-1] = np.random.randn(H) # start off the chain with random gradient
for t in reversed(range(T)):
    dss[t] = (hs[t] > 0) * dhs[t] # backprop through the nonlinearity
    dhs[t-1] = np.dot(Whh.T, dss[t]) # backprop into previous hidden state